In [19]:
import argparse
import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    TaskType,
)

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm


import warnings

warnings.filterwarnings("ignore")

In [20]:
import numpy as np
import random

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(114514)  # 设置固定随机种子

In [21]:
torch.cuda.empty_cache()

In [22]:
device = "cuda"
batch_size = 16
model_name_or_path = "FacebookAI/roberta-large"
task = "mrpc"
peft_type = PeftType.LORA
num_epochs = 20
lr = 3e-4

In [23]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

datasets = load_dataset("glue", task)
metric = evaluate.load("glue", task)


def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, max_length=512)
    return outputs


tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "sentence1", "sentence2"],
)

tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)

In [24]:
torch.cuda.empty_cache()

In [25]:
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
# target_modules = ["query", "value"]
chain_epoch = 5

In [26]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True, num_labels=2)
# model.config.problem_type = "regression"
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,838,082 || all params: 357,199,876 || trainable%: 0.5146


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A

In [27]:
# lambda_orth = 0.02
lambda_orth = 0.5
# lr_b = lambda_lr * lr_a
lambda_lr = 2.0
# 每隔 ？ step 计算一次正交性损失
orth_interval = 5  

In [28]:
# 获取LoRA参数
lora_params = {n: p for n, p in model.named_parameters() if 'lora' in n}
# 分组LoRA参数
param_groups = [
    {"params": [p for n, p in lora_params.items() if 'lora_A' in n], "lr": lr},  # A矩阵的学习率
    {"params": [p for n, p in lora_params.items() if 'lora_B' in n], "lr": lambda_lr * lr},  # B矩阵的学习率
]

beta1 = 0.9
beta2 = 0.99

optimizer = AdamW(param_groups, betas=(beta1, beta2))

#optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * chain_epoch),
    num_training_steps=(len(train_dataloader) * chain_epoch),
)

In [29]:
saved_lora_a_matrices = []  # 用于保存之前训练的 LoRA 模块的 A 矩阵
saved_lora_b_matrices = []  # 用于保存之前训练的 LoRA 模块的 B 矩阵

def extract_lora_a_matrices(model):
    """提取 LoRA 的 A 矩阵"""
    lora_matrices = []
    for name, param in model.named_parameters():
        if "lora_A" in name:  # 识别 LoRA 的 A 矩阵
            lora_matrices.append(param.detach().clone())  # 保存当前 A 矩阵
    return lora_matrices

def extract_lora_b_matrices(model):
    """提取 LoRA 的 B 矩阵"""
    lora_matrices = []
    for name, param in model.named_parameters():
        if "lora_B" in name:  # 识别 LoRA 的 B 矩阵
            lora_matrices.append(param.detach().clone())  # 保存当前 B 矩阵
    return lora_matrices

In [30]:
def orthogonality_loss(current_matrices, saved_matrices):
    """计算当前 LoRA 和之前所有 LoRA 的正交性损失"""
    loss = 0
    for B_t in current_matrices:
        for B_i in saved_matrices:
            # 计算 B_t 和 B_i 的正交性
            product = torch.matmul(B_t.T, B_i)  # 矩阵乘积
            loss += torch.norm(product, p="fro") # Frobenius 范数
    return loss


In [31]:
import re

def apply_shared_lora_a_matrices(model, saved_lora_a_matrices):
    """
    将保存的 LoRA A 矩阵应用到当前模型中，并冻结这些参数。
    """
    lora_idx = 0
    for name, param in model.named_parameters():
        if 'lora_A' in name:  #  A 矩阵的名称包含 'lora_A'
            # 提取层和注意力头的信息，例如 "encoder.layer.0.attention.self.query.lora_A"
            layer_match = re.match(r'.*encoder.layer.(\d+)\.attention.self\.(query|value)\.lora_A', name)
            if layer_match:
                layer_num = int(layer_match.group(1))
                if layer_num < len(saved_lora_a_matrices) // 2:
                    shared_a = saved_lora_a_matrices[lora_idx]
                    param.data = shared_a.data.clone() 
                    param.requires_grad = False
                    print(f"Applied shared LoRA A matrix for {name}")
                    
                    lora_idx += 1
                    

In [32]:
print(torch.initial_seed())

114514


In [33]:
model.to(device)
for epoch in range(num_epochs):
    if epoch != 0 and epoch % chain_epoch == 0:
        # 第一组lora的 A 矩阵作为共享 A 矩阵
        # epoch == chain_epoch // epoch % chain_epoch == 0
        if epoch == chain_epoch:
            saved_lora_a_matrices = extract_lora_a_matrices(model)
            print(f"Saved LoRA A matrices at epoch {epoch}")
        
        # 提取当前 LoRA 的 B 矩阵
        current_lora_b_matrices = extract_lora_b_matrices(model)
        # 将当前 B 矩阵保存到列表中
        saved_lora_b_matrices.extend(current_lora_b_matrices)
        orth_loss = 0.0
        
        # 合并
        model.merge_and_unload()
        peft_config.r = 8
        
        # 拓展
        model = get_peft_model(model, peft_config)
        # 共享 A
        if saved_lora_a_matrices:
            apply_shared_lora_a_matrices(model, saved_lora_a_matrices)
            # for name, param in model.named_parameters():
            #     print(f"{name}: requires_grad={param.requires_grad}")
            for name, param in model.named_parameters():
                if param.requires_grad:
                    print(f"Active parameter: {name}")

        
        #重新实例化优化器
        # 获取LoRA参数
        lora_params = {n: p for n, p in model.named_parameters() if 'lora' in n}
        # 分组LoRA参数
        param_groups = [
            {"params": [p for n, p in lora_params.items() if 'lora_A' in n], "lr": lr},  # A矩阵的学习率
            {"params": [p for n, p in lora_params.items() if 'lora_B' in n], "lr": lambda_lr * lr},  # B矩阵的学习率
        ]
        optimizer = AdamW(param_groups, betas=(beta1, beta2))
        # optimizer = AdamW(params=model.parameters(), lr=lr)
        lr_scheduler = get_linear_schedule_with_warmup(
            optimizer=optimizer,
            num_warmup_steps=0.06 * (len(train_dataloader) * chain_epoch),
            num_training_steps=(len(train_dataloader) * chain_epoch),
        )
        print(f"new lora, r = {peft_config.r}:")
        torch.cuda.empty_cache()
        
        
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        
        loss = outputs.loss
        # task_loss = outputs.loss
        if saved_lora_b_matrices and step % orth_interval == 0 and step != 0:
            current_lora_b_matrices = extract_lora_b_matrices(model)
            orth_loss = orthogonality_loss(current_lora_b_matrices, saved_lora_b_matrices)
        else:
            orth_loss = 0.0
        # 总损失
        loss = loss + lambda_orth * orth_loss
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        # predictions = outputs.logits.squeeze().cpu().numpy()
        # references = batch["labels"].cpu().numpy() 
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|██████████| 26/26 [00:00<00:00, 35.09it/s]


epoch 0: {'accuracy': 0.7867647058823529, 'f1': 0.8603531300160514}


100%|██████████| 26/26 [00:00<00:00, 36.25it/s]


epoch 1: {'accuracy': 0.8578431372549019, 'f1': 0.9013605442176871}


100%|██████████| 26/26 [00:00<00:00, 36.48it/s]


...

epoch 17: {'accuracy': 0.8970588235294118, 'f1': 0.926056338028169}


100%|██████████| 26/26 [00:00<00:00, 37.43it/s]


epoch 18: {'accuracy': 0.8995098039215687, 'f1': 0.9281961471103327}


100%|██████████| 26/26 [00:00<00:00, 37.24it/s]

epoch 19: {'accuracy': 0.8946078431372549, 'f1': 0.9244288224956063}
